In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import random_split, DataLoader, Dataset
from PIL import Image
import os
import numpy as np
from pyts.image import GramianAngularField

In [69]:
class StockCandlestickDataset(Dataset):
    def __init__(self, data_dir, transform=None, gaf_image_size=224):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.endswith(".png")
        ]
        
        # Set up GAF transformer
        self.gaf_transformer = GramianAngularField(image_size=gaf_image_size, method='summation')

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        
        # Load the image as grayscale and convert it to a NumPy array
        image = Image.open(img_path).convert('L')
        image_array = np.array(image)
        
        # Perform GAF transformation
        gaf_image = self.gaf_transformer.fit_transform(image_array.reshape(1, -1))[0]

        # Convert GAF image back to PIL format for further transformations
        gaf_image = Image.fromarray((gaf_image * 255).astype(np.uint8)).convert("RGB")
        
        # Apply further transformations (e.g., resizing and normalization)
        if self.transform:
            gaf_image = self.transform(gaf_image)
        
        # Determine label based on filename
        label = 1 if "Bullish" in img_path else 0

        return gaf_image, label

In [70]:
# Define image transformations including VGG-16 preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # VGG-16 normalization
])

data_dir = 'D:\\codyb\\COMP6970_Final_Project_Data\\charts\\20_minute_intervals'

# Create dataset with GAF transformation
dataset = StockCandlestickDataset(data_dir, transform=transform)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
class CandlestickClassifier(nn.Module):
    def __init__(self):
        super(CandlestickClassifier, self).__init__()
        # Load VGG-16 as a feature extractor, exclude its fully connected layers
        self.vgg_base = models.vgg16().features
        for param in self.vgg_base.parameters():
            param.requires_grad = False  # Freeze VGG-16 layers

        # Custom layers with reduced max pooling and adaptive pooling for consistent size
        self.conv_layers = nn.Sequential(
            nn.Conv2d(512, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4))  # Output fixed to 4x4 for consistency
        )

        # Calculate flattened size (32 * 4 * 4 = 512)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 4 * 4, 128),  # Adjust input size here based on adaptive pooling output
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Sigmoid for binary classification
        )

    def forward(self, x):
        x = self.vgg_base(x)
        x = self.conv_layers(x)
        x = self.fc(x)
        return x

In [72]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device being used:", device)

PyTorch version: 2.5.1+cu118
CUDA available: True
CUDA version: 11.8
Number of GPUs: 1
GPU 0: NVIDIA GeForce GTX 1660 Ti
Device being used: cuda


In [73]:
# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CandlestickClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

c:\Users\codyb\miniconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\codyb\miniconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.6916
Epoch [2/10], Loss: 0.6902
Epoch [3/10], Loss: 0.6883
Epoch [4/10], Loss: 0.6838
Epoch [5/10], Loss: 0.6741
Epoch [6/10], Loss: 0.6533
Epoch [7/10], Loss: 0.6186
Epoch [8/10], Loss: 0.5768
Epoch [9/10], Loss: 0.5399
Epoch [10/10], Loss: 0.4910
Test Accuracy: 70.00%


# Accuracies
- Bullish/Bearish
    - With GAF
        - 70%
    - Without GAF
        - 92.5%
- Candlestick Patterns